In [1]:
from openai import OpenAI
import json
import uuid
import chromadb

# create an OpenAI client
client = OpenAI()

#create a chromaDB client
chroma_client = chromadb.PersistentClient()

collection = chroma_client.get_or_create_collection(name="Real_Estate" , metadata={"hnsw:space": "cosine"} )

# Check if there is any existing data in the collection



In [2]:
collection.count()

0

In [3]:
# Define the function that will injects the list into the chroma DB
def generate_listing(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description):
    print(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description)
    unique_id = str(uuid.uuid4())
    collection.upsert(
    documents=[
        Description,Neighborhood_Description
    ],
    metadatas=[{"Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size},{"Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size}],
    ids=[f"{unique_id}_Description", f"{unique_id}_Neighborhood_Description"]
)


available_functions = {
        "generate_listing": generate_listing,
    }

In [4]:
messages = [{"role": "user", "content": "create 15 sample real estate listings"}]
tools=[
    {
        "type": "function",
        "function": {
            "name": "generate_listing",
            "description": "Create a real estate Listing",
            "parameters": {
                "type": "object",
                "properties": {
                    "Neighborhood": {
                        "type": "string",
                        "description": "e.g. Green Oaks"
                    },
                    "Price": {
                        "type": "string",
                        "description": "e.g. $800,000"
                    },
                    "Bedrooms": {
                        "type": "number",
                        "description": "e.g. 2"
                    },
                    "Bathrooms": {
                        "type": "number"
                    },
                    "House Size": {
                        "type": "string",
                        "description": "e.g. 2,000 sqft"
                    },
                    "Description": {
                        "type": "string",
                        "description": "e.g. Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem."
                    },
                    "Neighborhood Description": {
                        "type": "string",
                        "description": "e.g. Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze"
                    }
                },
                "required": ["Neighborhood","Price","Bedrooms","Bathrooms","House Size","Description","Neighborhood Description"]
            }
        }
    }
    ]

response = client.chat.completions.create(
        model="gpt-4o", #gpt 4o is accurate in calling the exact number of calls based on request. gpt-3-turbo is not fully accurate. While the ideal way may be to call in loop. Since I am lazy, switched to gpt4o. Slightly higher cost.
        messages=messages,
        tools=tools,
        tool_choice="required",  # Forcing the use of tools
    )
response_message = response.choices[0].message
tool_calls = response_message.tool_calls



In [5]:
tool_calls

[ChatCompletionMessageToolCall(id='call_Z59KwnBGhTk8QqsGZSV111Cg', function=Function(arguments='{"Neighborhood": "Maple Grove", "Price": "$650,000", "Bedrooms": 3, "Bathrooms": 2, "House Size": "1,800 sqft", "Description": "Discover comfortable living in this 3-bedroom, 2-bathroom home in Maple Grove. Featuring spacious rooms, modern appliances, and a large backyard, this property is perfect for families. Enjoy the tranquility of Maple Grove with convenient access to nearby parks and schools. Don\'t miss the opportunity to call this charming house your home.", "Neighborhood Description": "Maple Grove is a family-friendly neighborhood known for its great schools, parks, and a strong sense of community. Residents enjoy access to Maple Grove Park and a variety of local shops and restaurants. It\'s an ideal place for families to grow and thrive."}', name='generate_listing'), type='function'),
 ChatCompletionMessageToolCall(id='call_07EP1QUsBzxuHgURyqzTeFJy', function=Function(arguments='{"

In [6]:
if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                Neighborhood=function_args.get("Neighborhood"),
                Price=function_args.get("Price"),
                Bedrooms=function_args.get("Bedrooms"),
                Bathrooms=function_args.get("Bathrooms"),
                House_Size=function_args.get("House Size"),
                Description=function_args.get("Description"),
                Neighborhood_Description=function_args.get("Neighborhood Description"),
            )
            print(function_response)

Maple Grove $650,000 3 2 1,800 sqft Discover comfortable living in this 3-bedroom, 2-bathroom home in Maple Grove. Featuring spacious rooms, modern appliances, and a large backyard, this property is perfect for families. Enjoy the tranquility of Maple Grove with convenient access to nearby parks and schools. Don't miss the opportunity to call this charming house your home. Maple Grove is a family-friendly neighborhood known for its great schools, parks, and a strong sense of community. Residents enjoy access to Maple Grove Park and a variety of local shops and restaurants. It's an ideal place for families to grow and thrive.
None
Pine Hills $720,000 4 3 2,200 sqft Step into this stunning 4-bedroom, 3-bathroom home located in the scenic Pine Hills. The open floor plan and modern interior design make this the perfect place for both relaxation and entertaining. The master suite features a luxurious en-suite bathroom and a walk-in closet. The outdoor space includes a lovely patio and a wel

In [7]:
collection.count()

30